<h1>Final project of IBM Data Science Certification</h1>
<h2>Segmenting and Clustering Neighborhoods in Toronto</h2>
<h3>Problem 1 - Retrieve data from webpages</h3>
<h3>By: Aurelio Álvarez Ibarra</h3>

This notebook contains the code to scrape data from the <a href='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'>List of postal codes of Canada: M</a> to create a dataframe that contains <b>Neighborhoods</b> assigned to a <b>Borough</b> with individual <b>Postal codes</b>. The requirements for this problem are:
<ul>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
    <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
    <li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
    <li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</li>
</ul>

<h4>Install packages and import libraries</h4>
This notebook will use <code>BeautifulSoup4</code> to retrieve the data from Wikipedia, as well as <code>lxml</code> for parsing and <code>requests</code> to get the data.

In [1]:
# Download packages
!pip install beautifulsoup4 lxml

# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

<h4>Initializing the data retrieval</h4>
The following code retrieves the data from the desired URL (in text). It then creates a <code>BeautifulSoup</code> object that contains the code of the webpage.

In [2]:
# Define the target URL
myurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Retrieve data from URL
source = requests.get(myurl).text
# Create BSoup object
mysoup = BeautifulSoup(source,'lxml')

The specific block of information to be acquired (i.e. the table) is retrieved from the <code>BeautifulSoup</code> object. Uncomment the <code>print</code> statement to see the structure of the table.

In [3]:
# Copy important block from BSoup object
mytable = mysoup.find('table')
# Print table object (prettified) to analyze data important block from BSoup object
###print(mytable.prettify()) # Uncomment for long, skinny details...

According to the information in <code>mytable</code>, each row of the table is enclosed in <code>tr</code> tags (for Table Row). The headers of the table are enclosed in <code>th</code> tags (for Table Header). Finally, the data in each cell is enclosed in <code>td</code> tags (for Table Data). Let's see how it looks for a few examples:

In [4]:
mynum = 0
# The first loop does not consider table headers... they are fixed and I know them
for mytr in mytable.find_all('tr'): # Looping for each row in the table
    for mycell in mytr.find_all('td'): # Looping for each cell in the row
        print(mycell.text)
    mynum = mynum+1
    if (mynum>5):
        break

M1A

Not assigned



M2A

Not assigned



M3A

North York

Parkwoods

M4A

North York

Victoria Village

M5A

Downtown Toronto

Regent Park / Harbourfront



If you see a blank line in the previous output, it is because there is no data in the original table.

<h4>Retrieving data and saving into dataframe</h4>
So, now that I have the structure to retrieve data for some rows of the table, let's do it for the full table. Remember that the headers are fixed so there is no need to include them in the retrieving loop.

In [5]:
toronto_df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])
#
header = True
for mytr in mytable.find_all('tr'): # Looping for each row in the table
    # Initialize data list (row)
    data = []
    for mycell in mytr.find_all('td'): # Looping for each cell in the row
        data.append(mycell.text.strip()) # Strip removes the \n in the end of the cell data
        # The prenious line works for any number of columns (cells) in a row.
    # Write values from the row
    size = len(toronto_df) # Current size of dataframe
    if header: # The header row (which is only one) leaves "data" as a blank list!
        header = False
    else: # Non-header rows can be assigned to dataframe
        toronto_df.loc[size] = data # Appending data after last row of dataframe
#
# Check results
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


<h4>Cleaning dataframe</h4>
The following code will clean the dataframe according to requirements. Those are:
<ul>
    <li>Ignore cells with a <b>Not assigned</b> borough.</li>
    <li>Merge neighborhoods with the same Postal Code (separated by commas).</li>
    <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
</ul>

In [6]:
# List of boroughs with an assignment
condition1 = toronto_df['Borough']!='Not assigned'
tmp1 = toronto_df[condition1]
tmp1 = tmp1.reset_index(drop=True) # Drops the old index column
tmp1

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [7]:
# Copying borough name to neighborhood when neighborhood is not assigned
tmp2 = tmp1
for i,hood in enumerate(tmp1['Neighborhood']):
    if (hood=='Not assigned' or hood==''):
        bor = tmp2['Borough'][i]
        print('Updating Neighborhood name for ',bor,' in index ',i)
        tmp2['Neighborhood'][i] = bor
tmp2 = tmp2.reset_index(drop=True) # Drops the old index column
tmp2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [8]:
# Merge neighborhoods with the same PostalCode (separated by commas)
tmp3 = tmp2.groupby('PostalCode')['Neighborhood'].apply(','.join).reset_index()

# The previous statement successfully combines different neighborhoods with the same
#   postal code in one row, but only keeps the PostalCode and Neighborhood columns
#   thus the dataframe must be "rebuilt".

# Renaming the "new" (combined) neighborhood column to avoid confusion in the merging
tmp3.rename(columns={'Neighborhood':'Neighborhood_comb'},inplace=True)

# Merge original (tmp2) and new (combined, tmp3) dataframes (MySQL style)
merged = pd.merge(tmp2, tmp3, on='PostalCode')

# The previous statement adds the combined column to every duplicate PostalCode row. To
#   complete the cleaning, the "old" Neighborhood column will be dropped. After that, 
#   some rows will be identical in the dataframe, thus being ready to drop the
#   repeated rows in one shot.
merged.drop(['Neighborhood'],axis=1,inplace=True) # Dropping "old" Neighborhood column
merged.drop_duplicates(inplace=True) # Dropping duplicated rows

# Setting the correct name to the Neighborhood column
merged.rename(columns={'Neighborhood_comb':'Neighborhood'},inplace=True)

tmp3 = merged
tmp3

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


<b>NOTE</b>: It seems that at the time this notebook is created, some of the <i>issues</i> with the table were "solved". For example, all neighborhoods in a borough are assigned, and there are rows with several neighborhoods separated by forward slashes '/'. Thus, I will replace those slashes by commas as the exercise requires.

In [12]:
# Replacing / by , as the exercise required
dataframe = tmp3.replace(' / ', ', ',regex=True)
dataframe

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
print('After cleaning, the size of the dataframe is: ',dataframe.shape)

After cleaning, the size of the dataframe is:  (103, 3)
